# Introduction to Time Series Forecasting

***Summary***
- [Load data](#load-data) <br>
- [Preprocess data](#preprocess-data) <br>
- [Prediction](#prediction) <br>
- [Forecasting](#forecasting) <br>

In this lab we will use the dataset [Verkehrszählung MIV St. Gallen](https://stadt-stgallen.opendatasoft.com/explore/dataset/verkehrszahlung-miv-stadt-stgallen/information/?disjunctive.ort_id&disjunctive.bezeichnung&disjunctive.ri&disjunctive.ort_richtung_id) provided by [Open Data St. Gallen](https://stadt-stgallen.opendatasoft.com/pages/uber-uns/) to train and evaluate a simple classifier and perform time series forecasting.

We will start by loading the data from a csv into a [pandas](https://pandas.pydata.org/) dataframe. Utilizing pandas' rich set of functions we will then preprocess the raw data and prepare it for our purpose. Next, we visualize a subset of the data to identify distinctive properties and gain some inspiration for further processing. Based on this insights, we train a logistic regression model and various time series forecasting models.

In [ ]:
# Import libraries
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from google.colab import files
import io

<a id='load-data'></a>
## I. Load Data
This dataset can be downloaded as .csv-file under [this link](https://stadt-stgallen.opendatasoft.com/explore/dataset/verkehrszahlung-miv-stadt-stgallen/export/?disjunctive.ort_id&disjunctive.bezeichnung&disjunctive.ri&disjunctive.ort_richtung_id). Download the dataset and modify the following path accordingly.<br>
You can use the [head function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) to display the first five rows of the pandas dataframe.

In [ ]:
# Upload verkehrszahlung-miv-stadt-stgallen.csv as soon as `Choose Files` button appears
uploaded_data = files.upload()
df = pd.read_csv(io.BytesIO(uploaded_data['verkehrszahlung-miv-stadt-stgallen.csv']), sep=';')
df.head()

<a id='preprocess-data'></a>
## II. Preprocess data
This dataset comprises traffic counting data from 51 different locations for different directions.
As part of this preprocessing step, we will select the location/direction with the most data.
Then we will further reduce the amount of data and create a `df_predict` and a `df_forecast` that contain only the necessary data for the prediction task and the forecasting task, respectively.<br><br>
First, lets determine the number of unique values for each column with the following command.

In [ ]:
pd.DataFrame(df.nunique()).T

In order to determine the location/direction combination which has the most data samples we use the function [value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html).<br>
The [head(10)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) command displays the first ten rows of the resulting [Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html).

In [ ]:
df['ORT-RICHTUNG ID'].value_counts().head(10)

Next we determine the `ORT-RICHTUNG ID` of the location / direction combination with the most data samples by extracting the [first_valid_index](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.first_valid_index.html) of the sorted pandas series above.<br>
Then we extract a subset which only contains data samples of the corresponding `ORT-RICHTUNG ID`.
By executing [nunique](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nunique.html) we can verify that this subset really contains only one value for `STANDORT` and `RICHTUNG`.

In [ ]:
max_count_idx = df['ORT-RICHTUNG ID'].value_counts().first_valid_index()
max_count_idx

In [ ]:
df_sub_1 = df.loc[df['ORT-RICHTUNG ID']==max_count_idx]
df_sub_1.reset_index(drop=True, inplace=True)
df_sub_1.nunique().head(9)

In [ ]:
df_sub_1.head()

It turns out that we are analyzing the traffic volume at the `Herisauerstrasse 58`.

In [ ]:
street_name = df_sub_1['BEZEICHNUNG'].unique()
print(*street_name)

By creating a [pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html) we can determine the number of samples for each day of the week.

In [ ]:
df_sub_1.pivot_table(index='BEZEICHNUNG',columns='WOCHENTAG', values='ORT-RICHTUNG ID', aggfunc='count')

<a id='prediction'></a>
## III. Prediction
In this chapter we will use the data subset to train and test a very simple (and admittedly not very useful) classifier.
To this end, the `TAGESTOTAL` variable is the predictor and the qualitative `ARBEITSTAG` variable is the response which we will use to train a logistic regression model (which was introduced in lecture week 5).<br><br>
First, let's create a new dataframe called `df_predict` which contains only the data required for this classification task.
Next, we will visualize histogram of the data subset.

In [ ]:
df_predict = df_sub_1.loc[:,['ARBEITSTAG','TAGESTOTAL']]
df_predict.rename({'ARBEITSTAG':'y','TAGESTOTAL':'x'}, inplace=True, axis=1)
df_predict.head()

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(10,5))
sns.histplot(data=df_predict, x='x', hue='y', kde=True, palette=sns.color_palette('bright')[:2], ax=ax)

In the following we will:
- create a training and test dataset by using sklearn's [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function.
- instantiate and fit a [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model using the training split.
- evaluate the classifier's performance based on the test split, by using the score method.
- determine the decision boundary based on the trained model parameters `coef_` and `intercept_` and display this decision boundary in the histogram plot.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_predict['x'], df_predict['y'],
    stratify=df_predict['y'], test_size=0.33, random_state=42)


classifier = LogisticRegression(penalty='none')

In [ ]:
# Fit classifier to training data
classifier.fit(X_train[:,np.newaxis], y_train)

# Score classifier based on test set
score = classifier.score(X_test[:,np.newaxis], y_test)
print('Mean accuracy: {:4.3f}'.format(score))

In lecture 5 we've learned that a simple logistic regression model looks as follows:<br><br>
$\log(\frac{p(X)}{1-p(X)})=\beta_0 + \beta_1 \cdot X$<br><br>
During training, the parameters $\beta_0$ and $\beta_1$ are adjusted so that the model fits the training data as closely as possible.
These fitted parameters $\beta_0$ and $\beta_1$ can be accessed by the properties `intercept_` and `coef_`, respectively.
Since the decision boundary corresponds to the $X$ value for which
$\log(\frac{p(X)}{1-p(X)})=0$, the decision boundary can be calculated as follows:<br><br>
$X_b = -\frac{\beta_0}{\beta_1}$

In [ ]:
print(classifier.classes_)
print(classifier.coef_)
print(classifier.intercept_)

boundary = -classifier.intercept_ / classifier.coef_[0]

print(boundary)

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(10,5))
sns.histplot(data=df_predict, x='x', hue='y', kde=True, palette=sns.color_palette('bright')[:2], ax=ax)
ax.axvline(x=boundary, c='r')

<a id='forecasting'></a>
## IV. Forecasting
As part of this section, we will completely rearrange the dataframe `df_tmp_1` to obtain hourly traffic samples and create a `df_forecast` dataframe.
Based on this dataframe we'll attempt to make traffic forecasts for future time steps for which the correct values are not known.<br><br>

For this purpose we will test three different approaches:
- [Exponential Smoothing](https://www.statsmodels.org/stable/generated/statsmodels.tsa.holtwinters.SimpleExpSmoothing.html)
- [Triple Exponential Smoothing](https://www.statsmodels.org/dev/generated/statsmodels.tsa.holtwinters.ExponentialSmoothing.html)
- [Facebook Prophet](https://facebook.github.io/prophet/)

### Data preparation

In [ ]:
df_sub_1.head(1)

As you can see in the cell above, each row of the dataframe `df_sub_1` corresponds to the measurements of one day.
However, the traffic data is actually provided on an hourly basis.
The columns `1` to `24` contain the data for each hour of the corresponding day.<br>
Thus, we will rearrange the dataframe to have the datetime (day and hour) in the first column and the corresponding traffic value in the second column.<br><br>
To this end we will repeat each entry of the `DATUM` column 24 times and add the hour obtained by stacking the columns `1` to `24`.
The `df_forecast` results from adding these two dataframes.

In [ ]:
df_tmp_1 = df_sub_1['DATUM'].copy()
df_tmp_1 = pd.to_datetime(df_tmp_1)
df_tmp_1 = df_tmp_1.repeat(24).reset_index(drop=True)
df_tmp_1.head()

In [ ]:
df_tmp_2 = df_sub_1.loc[:,'1':'24'].stack().reset_index().drop('level_0', axis=1)
df_tmp_2.rename(columns={'level_1': 'ds', 0: 'y'}, inplace=True)
df_tmp_2['ds'] = df_tmp_2['ds'].astype('timedelta64[h]')
df_tmp_2.head()

In [ ]:
df_forecast = df_tmp_2.copy()
df_forecast['ds'] = df_tmp_1 + df_tmp_2['ds']
df_forecast.sort_values(by=['ds'], inplace=True)
df_forecast.head()

Next we plot this time series a by using the pandas' plot function.
The traffic development shows a distinctive pattern.
On weekends and at night, the traffic volume is significantly lower than at noon on a weekday.<br>
Next, we fit a time series model to these data.

In [ ]:
df_forecast.iloc[-1500:].plot(x='ds', y='y', title='Traffic at {}'.format(*street_name), xlabel='Time', ylabel='Number of Vehicles', figsize=(30,4))

### Exponential Smoothing
Exponential smoothing is a very simple forecasting method, where the estimated future sample corresponds to the weighted sum of the previous (known) samples.
The weight decreases exponentially the further in the past the sample is.<br>
The estimated value $s_t$ at time step $t$ is calculated as follows:<br><br>
$s_t = \alpha \cdot x_t + (1 - \alpha) \cdot s_{t-1} \qquad t > 0$<br><br>
For further theoretical information on exponential smoothing, see [here](https://en.wikipedia.org/wiki/Exponential_smoothing).<br><br>
In a first step we will use the data of the previous week in the dataset to fit an [SimpleExpSmoothing](https://www.statsmodels.org/stable/generated/statsmodels.tsa.holtwinters.SimpleExpSmoothing.html) model of the python package [statsmodels](https://www.statsmodels.org/stable/index.html).
Using this model, we attempt to predict the traffic for the next 24 hours in an one-hour interval.
Try to improve the predicted values by adjusting the function attributes.
Information on the function API can be found [here](https://www.statsmodels.org/stable/generated/statsmodels.tsa.holtwinters.SimpleExpSmoothing.html).

In [ ]:
df_forecast_exp = df_forecast.iloc[-168:].copy()
df_forecast_exp = df_forecast_exp.set_index('ds')
df_forecast_exp.index = pd.DatetimeIndex(df_forecast_exp.index.values,
                                         df_forecast_exp.index.inferred_freq)
df_forecast_exp.head()

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

model = SimpleExpSmoothing(df_forecast_exp).fit()

In [ ]:
# Forecast 100 weeks into the future
pred = model.forecast(24)

fig, ax = plt.subplots(1,1,figsize=(10,4))
ax.plot(pred.index, pred, c='r')
df_forecast_exp.iloc[-168:].plot(ax=ax)

### Triple Exponential Smoothing (TES)
Triple Exponential Smoothing (TES) (aka Hold-Winters algorithm) is an extension of the Simple Exponential Smoothing model to account for seasonality.
As the name already suggests, three components are estimated by using exponential smoothing.
Namely, the `level` component, `trend` component, and the `seasonality` component.<br><br>
For further theoretical information on TES see [here](https://en.wikipedia.org/wiki/Exponential_smoothing#Triple_exponential_smoothing_(Holt_Winters)).

In [ ]:
df_forecast_trip = df_forecast.iloc[-120:].copy()
df_forecast_trip = df_forecast_trip.set_index('ds')
df_forecast_trip.head()

In [ ]:
df_forecast_trip = df_forecast.iloc[-120:].copy()
df_forecast_trip = df_forecast_trip.set_index('ds')
df_forecast_trip.index = pd.DatetimeIndex(df_forecast_trip.index.values,
                                          df_forecast_trip.index.inferred_freq)
df_forecast_trip.head()

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

model = ExponentialSmoothing(df_forecast_trip, seasonal='add', seasonal_periods=24).fit(optimized = True)

# Forecast 48 hours out
pred = model.forecast(96)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,4))
ax.plot(pred, c='r')
df_forecast_trip.plot(ax=ax)
plt.show()

## Facebook Prophet

Prophet is a forecasting procedure for time series which is available as Python package.
It is a state-of-the-art forecasting method which works best with time series that have strong seasonal effects and several seasons of historical data.

Unlike most other methods, Prophet frames the forecast problem as a curve-fitting exercise, which is inherently different from time series models that consider the temporal dependency in the data (ARMA / ARIMA / SARIMA).
The time series model consists of three decomposable components:<br><br>
$y(t) = g(t) + s(t) + h(t) + \epsilon_t$<br><br>
$g(t)$ is the trend function, $s(t)$ represents the periodic changes (e.g. weekly and yearly seasonality) and $h(t)$ represents the effects of holidays.
This model can be considered as Generalized Additive Model (GAM), which will be introduced in semester week 10.<br><br>
If you want to learn more about GAM's, see 7.7 in the textbook.
If you want to learn more about Prophet, see [this paper](https://peerj.com/preprints/3190/).
More details on how to apply Prophet in Python can be found [here](https://facebook.github.io/prophet/docs/quick_start.html#python-api).

In [ ]:
#pip install pystan==2.19.1.1
#pip install prophet
from prophet import Prophet

model = Prophet(yearly_seasonality=True)

# model.fit(df_forecast.iloc[-17472:])
model.fit(df_forecast)

For a complete list of parameters and instructions on how to include the holyday component, see the [documentation](https://facebook.github.io/prophet/docs/quick_start.html).

In [ ]:
num_future = 168

future = model.make_future_dataframe(periods=num_future, freq='h')

In [ ]:
forecast = model.predict(future)

# Apply floor operator because number cannot be negative
forecast['yhat'] = forecast['yhat'].apply(lambda x: np.clip(x,0,None))
forecast['yhat_lower'] = forecast['yhat_lower'].apply(lambda x: np.clip(x,0,None))
forecast['yhat_upper'] = forecast['yhat_upper'].apply(lambda x: np.clip(x,0,None))

forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In case you have not installed `plotly` or it does not work, you can also display the result by using `Matplotlib`:<br><br>
`fig1 = model.plot(forecast)`<br>
`fig2 = model.plot_components(forecast)`

In [ ]:
# pip install ipywidgets
# pip install plotly
from prophet.plot import plot_plotly, plot_components_plotly
plot_plotly(model, forecast)

The advantage of splitting the model into different components is that the contribution of each component can be analyse separately.
This allows conclusions to be drawn about the development of the time series.
See below:

In [ ]:
plot_components_plotly(model, forecast)